In [1]:
%pylab qt

from matplotlib import animation
import datetime
from celestialbody import celestialbody
from celestialbody.celestialbody import CelestialBody
from celestialbody import display
import time

def var_param(N):
    x = np.linspace(-7,7,N//2)
    y = list(np.arctan(x)/np.pi)
    y += y[::-1]
    return 2*np.array(y)

def orbital_to_ecliptic_coordinates(x, y, omega_rad=0, Omega_rad=0, i_rad=0):
    X = (np.cos(omega_rad) * np.cos(Omega_rad) - np.sin(omega_rad) * np.sin(Omega_rad) * np.cos(i_rad)) * x \
        + (-np.sin(omega_rad) * np.cos(Omega_rad) - np.cos(omega_rad) * np.sin(Omega_rad) * np.cos(i_rad)) * y
    Y = (np.cos(omega_rad) * np.sin(Omega_rad) + np.sin(omega_rad) * np.cos(Omega_rad) * np.cos(i_rad)) * x \
        + (-np.sin(omega_rad) * np.sin(Omega_rad) + np.cos(omega_rad) * np.cos(Omega_rad) * np.cos(i_rad)) * y
    Z = (np.sin(omega_rad) * np.sin(i_rad)) * x + (np.cos(omega_rad) * np.sin(i_rad)) * y
    return X, Y, Z

def mysavefig(name):
    plt.tight_layout()
    plt.savefig(name, bbox_inches="tight", dpi=200)
    
############
# Sauvegarde
############
fps = 25
save_format = "gif" # None, "mp4" ou "gif"
dpi = 200

# writer
path = "images/"
if save_format == "mp4":
    Writer = animation.writers['ffmpeg']
elif save_format == "gif":
    # requiert ImageMagick : brew install imagemagick
    # make sure the full path for ImageMagick is configured
    # to find the path, type in console > which convert
    rcParams['animation.convert_path'] = r"/usr/local/bin/convert"
    Writer = animation.writers['imagemagick']
# sauvegarde
if save_format is None:
    pass
else:
    writer = Writer(fps=fps, metadata=dict(artist='Me'), bitrate=1800)

########################
# Animation parameters #
########################
f = .25
omega = 2*np.pi*f
duration = 1/f
N = int(duration * fps)

Populating the interactive namespace from numpy and matplotlib


# 6 animated plots for orbital parameters

The second one depends on the first one, so run them in the following order:

In [134]:
# Variable a circular orbit

scale = .25
offset = .75

fig = plt.figure(figsize=(3,2))
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
ax = fig.axes[0]
ax.set_aspect("equal")
plt.axis("off")
#plt.tight_layout()
plt.plot([0], [0], "o", color="gold")
c, = plt.plot([], [])
temp_text = ax.text(.8, 1.1, "")

var = var_param(N)
a = scale * var + offset

nu = np.linspace(0,2*np.pi, N)

def init():
    temp_text.set_text("")
    c.set_data([], [])
    return c

def animate(i):
    temp_text.set_text(r"$a$={:.2f}".format(a[i]))
    x = a[i] * np.cos(nu)
    y = a[i] * np.sin(nu)
    c.set_data(x, y)
    return c

anim = animation.FuncAnimation(fig, animate, frames=N, interval=1e3 / fps, init_func=init)

anim.save(path+"animation_a."+save_format, writer=writer, dpi=dpi)

In [135]:
# Variable e orbit

scale = .5
offset = .5

fig = plt.figure(figsize=(3,2))
plt.xlim(-2.1, 1.1)
plt.ylim(-1.1, 1.1)
ax = fig.axes[0]
ax.set_aspect("equal")
plt.axis("off")
plt.plot([0], [0], "o", color="gold")
c, = plt.plot([], [])
temp_text = ax.text(.8, 1.1, "")

var = var_param(N)
e = scale * var + offset

nu = np.linspace(0,2*np.pi, 10*N)

def init():
    temp_text.set_text("")
    c.set_data([], [])
    return c

def animate(i):
    temp_text.set_text(r"$e$={:.2f}".format(e[i]))
    rho = 1 / (1 + e[i] * np.cos(nu))
    a = (max(rho) + min(rho)) / 2
    x = rho * np.cos(nu) / a
    y = rho * np.sin(nu) / a
    c.set_data(x, y)
    return c

anim = animation.FuncAnimation(fig, animate, frames=N, interval=1e3 / fps, init_func=init)

anim.save(path+"animation_e."+save_format, writer=writer, dpi=dpi)

In [198]:
# Variable inclination orbit

scale = np.pi/8
offset = np.pi/8

fig = plt.figure(figsize=(3, 2))
sps = (1, 1)
ax = plt.subplot2grid(sps, (0, 0), projection='3d')

ax.set_xlim(-2.1, 1.1)
ax.set_ylim(-2.1, 1.1)
ax.set_zlim(-2.1, 1.1)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])
#plt.axis("off")

ax.plot([0], [0], "o", color="gold")
c, = ax.plot([], [], [], "-C0")
ax_x, = ax.plot([], [], [], "--k", alpha=.25)
ax_y, = ax.plot([], [], [], "--k", alpha=.25)
temp_text = ax.text(.8, 1.1, 1.1, "")

var = var_param(N)
nu = np.linspace(0,2*np.pi, 10*N)
e = .75

rho = 1 / (1 + e * np.cos(nu))
a = (max(rho) + min(rho)) / 2
x = 2*rho * np.cos(nu) / a
y = 2*rho * np.sin(nu) / a

inclination = offset + scale * var

def init():
    temp_text.set_text("")
    c.set_data([], [])
    return c

def animate(i):
    temp_text.set_text(r"$I$={:3.0f}$\degree$".format(inclination[i]*180/np.pi))
    X, Y, Z = orbital_to_ecliptic_coordinates(x,y, i_rad=inclination[i])
    c.set_data(X,Y)
    c.set_3d_properties(Z)
    X, Y, Z = orbital_to_ecliptic_coordinates(np.linspace(-4,2, 2),np.zeros(2), i_rad=inclination[i])
    ax_x.set_data(X,Y)
    ax_x.set_3d_properties(Z)
    X, Y, Z = orbital_to_ecliptic_coordinates(np.zeros(2), np.linspace(-2,2, 2), i_rad=inclination[i])
    ax_y.set_data(X,Y)
    ax_y.set_3d_properties(Z)
    return c

anim = animation.FuncAnimation(fig, animate, frames=N, interval=1e3 / fps, init_func=init)

anim.save(path+"animation_I."+save_format, writer=writer, dpi=dpi)

In [203]:
# Variable ascending node orbit

scale = np.pi/4
offset = np.pi/4

fig = plt.figure(figsize=(3, 2))
sps = (1, 1)
ax = plt.subplot2grid(sps, (0, 0), projection='3d')

ax.set_xlim(-2.1, 1.1)
ax.set_ylim(-2.1, 1.1)
ax.set_zlim(-2.1, 1.1)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])
#plt.axis("off")

ax.plot([0], [0], "o", color="gold")
c, = ax.plot([], [], [], "-C0")
ax_x, = ax.plot([], [], [], "--k", alpha=.25)
ax_y, = ax.plot([], [], [], "--k", alpha=.25)
temp_text = ax.text(.8, 1.1, 1.1, "")

var = var_param(N)
nu = np.linspace(0,2*np.pi, 10*N)
e = .75

rho = 1 / (1 + e * np.cos(nu))
a = (max(rho) + min(rho)) / 2
x = 2*rho * np.cos(nu) / a
y = 2*rho * np.sin(nu) / a

inclination = np.pi/8
Omega = offset + scale * var

def init():
    temp_text.set_text("")
    c.set_data([], [])
    return c

def animate(i):
    temp_text.set_text(r"$\Omega$={:3.0f}$\degree$".format(Omega[i]*180/np.pi))
    X, Y, Z = orbital_to_ecliptic_coordinates(x,y, i_rad=inclination, Omega_rad=Omega[i])
    c.set_data(X,Y)
    c.set_3d_properties(Z)
    X, Y, Z = orbital_to_ecliptic_coordinates(np.linspace(-4,2, 2),np.zeros(2), i_rad=inclination, Omega_rad=Omega[i])
    ax_x.set_data(X,Y)
    ax_x.set_3d_properties(Z)
    X, Y, Z = orbital_to_ecliptic_coordinates(np.zeros(2), np.linspace(-2,2, 2), i_rad=inclination, Omega_rad=Omega[i])
    ax_y.set_data(X,Y)
    ax_y.set_3d_properties(Z)
    return c

anim = animation.FuncAnimation(fig, animate, frames=N, interval=1e3 / fps, init_func=init)

anim.save(path+"animation_ascending_node."+save_format, writer=writer, dpi=dpi)

In [213]:
# Variable argument of perihelion orbit

scale = np.pi/4
offset = scale

fig = plt.figure(figsize=(3, 2))
sps = (1, 1)
ax = plt.subplot2grid(sps, (0, 0), projection='3d')

ax.set_xlim(-2.1, 1.1)
ax.set_ylim(-2.1, 1.1)
ax.set_zlim(-2.1, 1.1)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])
#plt.axis("off")

ax.plot([0], [0], "o", color="gold")
c, = ax.plot([], [], [], "-C0")
ax_x, = ax.plot([], [], [], "--k", alpha=.25)
ax_y, = ax.plot([], [], [], "--k", alpha=.25)
temp_text = ax.text(.8, 1.1, 1.1, "")

var = var_param(N)
nu = np.linspace(0,2*np.pi, 10*N)
e = .75

rho = 1 / (1 + e * np.cos(nu))
a = (max(rho) + min(rho)) / 2
x = 2*rho * np.cos(nu) / a
y = 2*rho * np.sin(nu) / a

Omega = np.pi/4
omega = offset + scale * var

def init():
    temp_text.set_text("")
    c.set_data([], [])
    return c

def animate(i):
    temp_text.set_text(r"$\omega$={:3.0f}$\degree$".format(omega[i]*180/np.pi))
    X, Y, Z = orbital_to_ecliptic_coordinates(x,y, i_rad=inclination, Omega_rad=Omega, omega_rad=omega[i])
    c.set_data(X,Y)
    c.set_3d_properties(Z)
    X, Y, Z = orbital_to_ecliptic_coordinates(np.linspace(-4,2, 2),np.zeros(2), i_rad=inclination, Omega_rad=Omega)
    ax_x.set_data(X,Y)
    ax_x.set_3d_properties(Z)
    X, Y, Z = orbital_to_ecliptic_coordinates(np.zeros(2), np.linspace(-2,2, 2), i_rad=inclination, Omega_rad=Omega)
    ax_y.set_data(X,Y)
    ax_y.set_3d_properties(Z)
    return c

anim = animation.FuncAnimation(fig, animate, frames=N, interval=1e3 / fps, init_func=init)

anim.save(path+"animation_argument_periapsis."+save_format, writer=writer, dpi=dpi)

In [225]:
# Variable mean anomaly orbit

scale = np.pi/16
offset = 9*np.pi/8

fig = plt.figure(figsize=(6, 4))
sps = (1, 1)
ax = plt.subplot2grid(sps, (0, 0), projection='3d')

ax.set_xlim(-2.1, 1.1)
ax.set_ylim(-2.1, 1.1)
ax.set_zlim(-2.1, 1.1)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])

ax.plot([0], [0], "o", color="gold")
c, = ax.plot([], [], [], "-C0")
pos, = ax.plot([], [], [], "oC0")
ax_x, = ax.plot([], [], [], "--k", alpha=.25)
ax_y, = ax.plot([], [], [], "--k", alpha=.25)
temp_text = ax.text(.8, 1.1, 1.1, "")

var = var_param(N)
nu = np.linspace(0,2*np.pi, 10*N)
e = .75

rho = 1 / (1 + e * np.cos(nu))
a = (max(rho) + min(rho)) / 2
x = 2*rho * np.cos(nu) / a
y = 2*rho * np.sin(nu) / a

omega = np.pi/3
M = scale * var + offset

X, Y, Z = orbital_to_ecliptic_coordinates(x,y, i_rad=inclination, Omega_rad=Omega, omega_rad=omega)
c.set_data(X,Y)
c.set_3d_properties(Z)
X, Y, Z = orbital_to_ecliptic_coordinates(np.linspace(-3,2, 2),np.zeros(2), i_rad=inclination, Omega_rad=Omega)
ax_x.set_data(X,Y)
ax_x.set_3d_properties(Z)
X, Y, Z = orbital_to_ecliptic_coordinates(np.zeros(2), np.linspace(-2,2, 2), i_rad=inclination, Omega_rad=Omega)
ax_y.set_data(X,Y)
ax_y.set_3d_properties(Z)

def init():
    pos.set_data([], [])
    return c

def animate(i):
    rho = 1 / (1 + e * np.cos(M[i]))
    x = 2*rho * np.cos(M[i]) / a
    y = 2*rho * np.sin(M[i]) / a
    X, Y, Z = orbital_to_ecliptic_coordinates(x,y, i_rad=inclination, Omega_rad=Omega, omega_rad=omega)
    pos.set_data([X], [Y])
    pos.set_3d_properties([Z])
    return pos

anim = animation.FuncAnimation(fig, animate, frames=N, interval=1e3 / fps, init_func=init)

anim.save(path+"animation_mean_anomaly."+save_format, writer=writer, dpi=dpi)

# First slide demo gif

In [52]:
# time recquired 14.7 minutes

t0 = time.time()

step  = .5
N = int(365.25 / step)
start = datetime.datetime.today()
stop  = start+datetime.timedelta(days=N*step)

# FIGURE SHAPE
fig = plt.figure(figsize=(16,8))
sps = (1,2)
ax1 = plt.subplot2grid(sps, (0,0))
ax2 = plt.subplot2grid(sps, (0,1), projection='3d')

# LISTS OF DEPICTED BODIES
planets = ["Mercury", "Venus", "Earth", "Mars", "Jupiter"]
asteroids = ["Ceres", "Pallas", "Juno", "Vesta"]
comets = ["Churyumov-Gerasimenko", "Borrelly", "Holmes"]
names = planets+asteroids+comets
categories = ["planet" for planet in planets] + ["asteroid" for asteroid in asteroids] + ["comet" for comet in comets]
#colormap = plt.cm.viridis
#colorst = [colormap(i) for i in np.linspace(0, 1, len(names))]

# COMPUTE SUCCESIVE POSITIONS
positions = []
for name, category in zip(names, categories):
    body = CelestialBody(name, category=category)
    pos = body.data("position", start=start, stop=stop, step=step)
    positions.append(pos)

# ANIMATED OBJECTS CREATION
positions1, temp_texts1, trajectories1 = [], [], []
positions2, temp_texts2, trajectories2 = [], [], []
for name, category, i in zip(names, categories, range(len(names))):   
    color = "C"+str(i)
    if category == "planet":
        marker = "o"
        linewidth = .5
        alpha = 1
        fontsize = "medium"
    elif category == "asteroid":
        marker = "."
        linewidth = .2
        alpha = 1
        fontsize = "small"
    elif category == "comet":
        marker = "*"
        linewidth = .2
        alpha = 1
        fontsize = "small"
        
    position1,   = ax1.plot([], [], marker=marker, color=color, alpha=alpha)
    trajectory1, = ax1.plot([], [], color=color, linewidth=linewidth, alpha=alpha)
    temp_text1 = ax1.annotate("", (0,0), color=color, textcoords="offset points", xytext=(2,2),
                              fontsize=fontsize, horizontalalignment='left', verticalalignment='bottom', alpha=alpha)
    
    position2,   = ax2.plot([], [], [], marker=marker, color=color, label=body.fullname, alpha=alpha)
    trajectory2, = ax2.plot([], [], color=color, linewidth=linewidth, alpha=alpha)
    temp_text2 = ax2.text(0,0,0,  "", fontsize=fontsize,  color=color, horizontalalignment='left', verticalalignment='bottom', alpha=alpha)

    positions1.append(position1)
    trajectories1.append(trajectory1)
    temp_texts1.append(temp_text1)
    positions2.append(position2)
    trajectories2.append(trajectory2)
    temp_texts2.append(temp_text2)

def init():
    # NON-ANIMATED SUN
    ax1.scatter(0,0, marker="o", color="gold")
    ax1.annotate("Sun", (0,0), color="gold", textcoords="offset points", xytext=(2,2),
                 horizontalalignment='left', verticalalignment='bottom', alpha=1)
    ax2.scatter(0,0,0, marker="o", color="gold")
    ax2.text(0,0,0,  "Sun", fontsize=fontsize,  color="gold",
             horizontalalignment='left', verticalalignment='bottom', alpha=1)

    for i, name in enumerate(names):
        category = categories[i]
        position = positions[i]
        position1   = positions1[i]
        trajectory1 = trajectories1[i]
        temp_text1  = temp_texts1[i]
        position2   = positions2[i]
        trajectory2 = trajectories2[i]
        temp_text2  = temp_texts2[i]
        
        body = CelestialBody(name, category=category)
        body.date = start
        x,y,z = position[0]
        X,Y,Z = body.orbit
        
        position1.set_data([x], [y])
        trajectory1.set_data(X, Y)
        temp_text1.set_text(body.fullname)
        temp_text1.set_position((x,y))
        temp_text1.xy = (x,y)
        position2.set_data([x], [y])
        position2.set_3d_properties([z])
        trajectory2.set_data(X, Y)
        trajectory2.set_3d_properties(Z)
        #temp_text2.set_text(body.fullname)
        #temp_text2.set_position((x,y))
        #temp_text2.xy = (x,y)
    return
    
        
def animate(i_frame):
    for i, name in enumerate(names):
        category = categories[i]
        position = positions[i]
        position1   = positions1[i]
        trajectory1 = trajectories1[i]
        temp_text1  = temp_texts1[i]
        position2   = positions2[i]
        trajectory2 = trajectories2[i]
        temp_text2  = temp_texts2[i]
        
        x,y,z = position[i_frame]
        position1.set_data([x], [y])
        temp_text1.set_position((x,y))
        temp_text1.xy = (x,y)
        position2.set_data([x], [y])
        position2.set_3d_properties([z])
    return positions1 + positions2

if True:
    a_lim = 6
    ax1.set_aspect("equal")
    ax1.set_xlim(-a_lim,a_lim)
    ax1.set_ylim(-a_lim,a_lim)

    ax2.set_xlim(-a_lim/2,a_lim/2)
    ax2.set_ylim(-a_lim/2,a_lim/2)
    ax2.set_zlim(-a_lim/2,a_lim/2)
    
    for ax in [ax1, ax2]:
        ax.set_xlabel("X (au)")
        ax.set_ylabel("Y (au)")
    ax2.set_zlabel("Z (au)")

anim = animation.FuncAnimation(fig, animate, frames=N, interval=1e3 / fps, init_func=init)

anim.save(path+"animation_demo."+save_format, writer=writer, dpi=dpi)

duration = time.time() - t0
print(duration/60)

14.673323114713034


# Ptolemy vs Copernicus

In [79]:
# takes roughly 5.45 minutes to run if this animation is saved!
# change step to 50 instead of 2 to check if eveyrthing goes well or don't save
# rule of thumb: 5.45 min needed to save a gif with 1461 frames
#                0.0037 min or 0.22 s per frame
#                0.093 min or 5.6 s per second of gif

t0 = time.time()

start       = datetime.datetime(2021, 1, 1)
stop        = datetime.datetime(2029, 1, 1)
step        = 2
names       = ["Earth", "Venus"]
ref         = "Sun"

show_speed=True
show_accel=False

names, coord_sun, coord_ref, dates = display.prepare_data(names, ref=ref, start=start, stop=stop, step=step)

fig = plt.figure(figsize=(4, 4))
ax = plt.subplot2grid((1,1), (0, 0))

Xs, Ys, Zs = coord_ref

for X, Y in zip(Xs, Ys):
    ax.plot(X,Y)
xlim, ylim = ax.get_xlim(), ax.get_ylim()
ax.clear()

positions, trajectories, speeds, accels = [], [], [], []
for i, name in enumerate(names[:-1]):
    if name == "Sun":
        label = "Soleil"
        color = "gold"
    elif name == "Earth":
        label = "Terre"
        color = "C"+str(i)
    elif name == "Venus":
        label = "Vénus"
        color = "C"+str(i)
    position,   = ax.plot([], [], "o", color=color, label=label)
    trajectory, = ax.plot([], [], "-", color=color, linewidth=.5)
    speed   = ax.quiver(0, 0, 0, 0, color=color, alpha=.5, width=.005, angles='xy', scale_units='xy', scale=1)
    accel   = ax.quiver(0, 0, 0, 0, color="k", alpha=.5, width=.005, angles='xy', scale_units='xy', scale=1)
    positions.append(position)
    trajectories.append(trajectory)
    speeds.append(speed)
    accels.append(accel)

def init():
    for position, trajectory in zip(positions, trajectories):
        position.set_data([], [])
        trajectory.set_data([], [])
    return

def animate(i):
    for X, Y, position, trajectory, speed, accel in zip(Xs, Ys, positions, trajectories, speeds, accels):
        ax.set_title(dates[0][i].date())
        position.set_data(X[i], Y[i])
        trajectory.set_data(X[:i], Y[:i])
        if show_speed:
            scale = 10
            u, v = (X[i + 1] - X[i]) / step, (Y[i + 1] - Y[i]) / step
            speed.set_UVC(scale * u, scale * v)
            speed.set_offsets((X[i], Y[i]))
        if show_accel:
            scale = 500
            u_v2, v_v2 = (X[i + 2] - X[i+1]) / step, (Y[i + 2] - Y[i+1]) / step
            u_v1, v_v1 = (X[i + 1] - X[i]) / step, (Y[i + 1] - Y[i]) / step
            u, v   = (u_v2 - u_v1) / step, (v_v2-v_v1) / step
            accel.set_UVC(scale * u, scale * v)
            accel.set_offsets((X[i], Y[i]))
    return positions + trajectories + speeds + accels

ax.legend(loc="upper left")
ax.set_aspect("equal")
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_xlabel("X (au)")
ax.set_ylabel("Y (au)")

n_unused_frames = 0
if show_speed:
    n_unused_frames = 1
if show_accel:
    n_unused_frames = 2
anim = animation.FuncAnimation(fig, animate, frames=len(Xs[0]) - n_unused_frames, interval=1e3 / fps, init_func=init)

#anim.save(path+"animation_."+save_format, writer=writer, dpi=dpi)

duration = time.time() - t0
print(duration/60)

0.0041338046391805015


# Acceleration

In [103]:
# takes roughly 5.45 minutes to run if this animation is saved!
# change step to 50 instead of 2 to check if eveyrthing goes well or don't save
# rule of thumb: 5.45 min needed to save a gif with 1461 frames
#                0.0037 min or 0.22 s per frame
#                0.093 min or 5.6 s per second of gif

t0 = time.time()

start       = datetime.datetime.today()
stop        = start+datetime.timedelta(days=2355)
step        = 5
names       = ["Churyumov-Gerasimenko"]
ref         = "Sun"

show_speed=True
show_accel=True

names, coord_sun, coord_ref, dates = display.prepare_data(names, ref=ref, start=start, stop=stop, step=step)

fig = plt.figure(figsize=(4, 4))
ax = plt.subplot2grid((1,1), (0, 0))

Xs, Ys, Zs = coord_ref

for X, Y in zip(Xs, Ys):
    ax.plot(X,Y)
xlim, ylim = ax.get_xlim(), ax.get_ylim()
ax.clear()

positions, trajectories, speeds, accels = [], [], [], []
for i, name in enumerate(names[:-1]):
    if name == "Sun":
        label = "Soleil"
        color = "gold"
    elif name == "Churyumov-Gerasimenko":
        label = "Tchouri"
        color = "k"
    position,   = ax.plot([], [], "o", color=color, label=label)
    trajectory, = ax.plot([], [], "-", color=color, linewidth=.5)
    speed   = ax.quiver(0, 0, 0, 0, color=color, alpha=.5, width=.005, angles='xy', scale_units='xy', scale=1)
    accel   = ax.quiver(0, 0, 0, 0, color=color, alpha=1, width=.005, angles='xy', scale_units='xy', scale=1)
    positions.append(position)
    trajectories.append(trajectory)
    speeds.append(speed)
    accels.append(accel)

def init():
    for position, trajectory in zip(positions, trajectories):
        position.set_data([], [])
        trajectory.set_data([], [])
    return

def animate(i):
    for X, Y, position, trajectory, speed, accel in zip(Xs, Ys, positions, trajectories, speeds, accels):
        ax.set_title(dates[0][i].date())
        position.set_data(X[i], Y[i])
        trajectory.set_data(X, Y)
        if show_speed:
            scale = 75
            u, v = (X[i + 1] - X[i]) / step, (Y[i + 1] - Y[i]) / step
            speed.set_UVC(scale * u, scale * v)
            speed.set_offsets((X[i], Y[i]))
        if (i>0) and show_accel:
            scale = 30000
            u_v2, v_v2 = (X[i + 1] - X[i]) / step, (Y[i + 1] - Y[i]) / step
            u_v1, v_v1 = (X[i] - X[i - 1]) / step, (Y[i] - Y[i - 1]) / step
            u, v   = (u_v2 - u_v1) / step, (v_v2-v_v1) / step
            accel.set_UVC(scale * u, scale * v)
            accel.set_offsets((X[i], Y[i]))
    return positions + trajectories + speeds + accels

ax.legend(loc="upper left")
ax.set_aspect("equal")
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_xlabel("X (au)")
ax.set_ylabel("Y (au)")

n_unused_frames = 0
if show_speed:
    n_unused_frames = 1
if show_accel:
    n_unused_frames = 2
anim = animation.FuncAnimation(fig, animate, frames=len(Xs[0]) - n_unused_frames, interval=1e3 / fps, init_func=init)

anim.save(path+"animation_accel."+save_format, writer=writer, dpi=dpi)

duration = time.time() - t0
print(duration/60)

1.7296407183011373
